## VaR  Méthode Paramétrique

In [1]:
#Importation des librairies nécessaires 

import numpy as np
import pandas as pd
import datetime as dt 
import yfinance as yf 
import matplotlib.pyplot as plt
from scipy.stats import norm

In [2]:
# Fixation une plage de temps pour un certain nombre d'années

années = 15

datefin = dt.datetime.now()

datedébut = datefin - dt.timedelta(days = 365 * années)

In [3]:
# Création d'une liste de tickers 

tickers = ['SPY', 'BND', 'GLD', 'QQQ', 'VTI']

In [4]:
# Cours de clôture ajustés quotidiennement pour les tickers  ( ajustés car ils prennent en compte les dividendes et stock splits)

cloture_aj_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start = datedébut, end = datefin)
    cloture_aj_df[ticker] = data['Adj Close']

    
print(cloture_aj_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                   SPY        BND         GLD         QQQ         VTI
Date                                                                 
2008-10-27   62.827511  47.649933   72.180000   25.096775   31.531334
2008-10-28   70.169250  47.281429   73.790001   27.869764   34.696533
2008-10-29   69.660339  47.624111   74.000000   27.799780   34.349876
2008-10-30   72.070137  47.307316   72.709999   28.727020   35.510445
2008-10-31   72.466797  47.281429   71.339996   28.770763   36.294205
...                ...        ...         ...         ...         ...
2023-10-16  436.040009  69.089996  177.960007  369.410004  216.009995
2023-10-17  436.019989  6

In [5]:
# calcul des rendements journaliers en log  et suppression de tous les NA

rendements_log  = np.log(cloture_aj_df/ cloture_aj_df.shift(1))
rendements_log  = rendements_log.dropna()

print(rendements_log)

                 SPY       BND       GLD       QQQ       VTI
Date                                                        
2008-10-28  0.110517 -0.007764  0.022060  0.104803  0.095658
2008-10-29 -0.007279  0.007222  0.002842 -0.002514 -0.010041
2008-10-30  0.034009 -0.006674 -0.017586  0.032810  0.033228
2008-10-31  0.005489 -0.000547 -0.019022  0.001522  0.021831
2008-11-03  0.002888  0.007796 -0.003370 -0.002131 -0.003328
...              ...       ...       ...       ...       ...
2023-10-16  0.010466 -0.005197 -0.004877  0.011243  0.011547
2023-10-17 -0.000046 -0.006535  0.001853 -0.003281  0.001295
2023-10-18 -0.013415 -0.004380  0.014367 -0.013205 -0.014905
2023-10-19 -0.008825 -0.004399  0.012199 -0.009401 -0.009477
2023-10-20 -0.012364  0.003668  0.002727 -0.015030 -0.012442

[3771 rows x 5 columns]


In [6]:
# Création d'un portefeuille équipondéré

valeur_portefeuille = 1000000
poids = np.array([1/len(tickers)] * len(tickers))
print(poids)

[0.2 0.2 0.2 0.2 0.2]


In [7]:
# Calcul des rendements historiques du portefeuille

rendements_historiques = (rendements_log * poids).sum(axis = 1)
print(rendements_historiques)

Date
2008-10-28    0.065055
2008-10-29   -0.001954
2008-10-30    0.015157
2008-10-31    0.001854
2008-11-03    0.000371
                ...   
2023-10-16    0.004637
2023-10-17   -0.001343
2023-10-18   -0.006308
2023-10-19   -0.003981
2023-10-20   -0.006688
Length: 3771, dtype: float64


In [8]:
#On retrouve les rendements historiques du jour X

jours = 5

range_rendements = rendements_historiques.rolling(window = jours).sum()


In [9]:
# Création d'une matrice de covariance pour tous les titres

matrice_cov = rendements_log.cov() * 252

In [10]:
# Calcul de l'écart type du portefeuille

ecart_type_port = np.sqrt(poids.T @ matrice_cov @ poids)

In [11]:
# Définition de différents niveaux de confiance pour visualiser

niveaux_conf = [0.9, 0.95, 0.99]

In [17]:
# Calcul de la vaR à différents niveaux de confiance

from scipy.stats import norm

VaRs = []
 
for nc in niveaux_conf:
    VaR = valeur_portefeuille * ecart_type_port * norm.ppf(nc) * np.sqrt(jours/252)
    VaRs.append(VaR)

In [32]:
# Affichage des résultats 

print("Les diffèrentes VaRs à 90%, 95% et 99% sont ", VaRs, "$")

Les diffèrentes VaRs à 90%, 95% et 99% sont  [22975.510470493497, 29488.748439391253, 41706.499664195915] $
